<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>

<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 250px; display: inline" alt="IMT"/> </a>
</center>

# Modèles linéaires et sélection de variables  en grande dimension avec <a href="https://cran.r-project.org/"><img src="https://cran.r-project.org/Rlogo.svg" style="max-width: 40px; display: inline" alt="R"/></a> : Etude de données d'expression de gènes

** Objectifs **

Sélection de variables en grande dimension dans le cadre du modèle linéaire. Le nombre de variables est ici très supérieur au nombre d'observations

Comparaison sur le même jeu de données des méthodes suivantes :

-  Algorithmes de sélection de modèles (critères AIC, BIC)
- Régularisation ( Lasso, Elastic Net)
- Régression sur composantes (PLS)


# Les données

Le jeu de  données `liver.toxicity`  contient, pour 64 individus :
- 10 variables cliniques parmi lesquelles nous retiendrons une variable à expliquer $Y$ :  niveau de toxicité du foie chez le rat (il s'agit d'un niveau de cholestérol)
-  3116 variables explicatives qui sont  des mesures d'expression de gènes (logarithme du rapport des niveaux d'expression entre deux conditions)


** Source : Bushel, P., Wolfinger, R. D. and Gibson, G. (2007). Simultaneous clustering of gene expression
data with clinical chemistry and pathological evaluations reveals phenotypic prototypes. BMC
Systems Biology.**

On est donc clairement ici dans un contexte de grande dimension : le nombre de variables est très supérieur au nombre d'individus. 

L'objectif est double : 
- Identifier les gènes qui ont un lien avec le niveau de cholestérol
- Construire un modèle permettant de faire de la prédiction. 


## Lecture des données

In [ ]:
library(mixOmics)
data(liver.toxicity)


#load("liverToxicity.rdata")

# décrit les données
gene=liver.toxicity$gene
dim(gene)
clinic=liver.toxicity$clinic
dim(clinic)
names(clinic)


nomrats=dimnames(gene)[[1]]
nomrats

In [ ]:
# On retient les mesures d'expression des gènes et la variable clinique associée au taux de cholestérol : 

cholesterol=clinic[,10]
liver=data.frame(cholesterol,gene)


## Description des données

In [ ]:

summary(liver[,1:5])

hist(cholesterol)


In [ ]:
hist(cor(gene,cholesterol))

In [ ]:
### Distribution de quelques covariables : 

par(mfrow=c(2,2))
hist(liver[,2],main="Histogramme",xlab="")
hist(liver[,3],main="Histogramme",xlab="")
hist(liver[,4],main="Histogramme",xlab="")
hist(liver[,5],main="Histogramme",xlab="")



In [ ]:
### Matrice de correlations complete 
cormat<-cor(liver)
dim(cormat)
max(abs(cormat[1,-1]))

In [ ]:
## Variables les plus corrélées à la réponse
which.max(abs(cormat[1,-1]))
cormat[1,1158]

colnames(liver)[1158]

which.max(abs(cormat[1,c(-1,-1158)]))
cormat[1,1837]

In [ ]:
## Graphes vs. la réponse correspondants
par(mfrow=c(1,2))
plot(liver[,1]~liver[,1158])
plot(liver[,1]~liver[,1837])


# Sélection de  variables
## Sélection par méthode  forward

In [ ]:
library(leaps)
#help(regsubsets)

modselect=regsubsets(cholesterol~.,data=liver,method="forward",nvmax=6,nbest=1,intercept=TRUE,really.big=TRUE)
attributes(modselect)
# afficher les variables selectionnees

coef(modselect,1:6)

### Selection forward: autre méthode pour afficher les variables selectionnees : pas très lisible ..
varselect1=summary(modselect)$which
varselect1

mod1<-lm(cholesterol~.,data=liver[,varselect1[6,]])
summary(mod1)



In [ ]:
selectforward=which(varselect1[6,]==TRUE)
attributes(selectforward)

## Sélection par méthode "sequential replacement"

In [ ]:
library(leaps)

modselect=regsubsets(cholesterol~.,data=liver,method="seqrep",nvmax=6,nbest=1,intercept=TRUE,really.big=TRUE)

# nbest : nombre de modèle de chaque taille affiché

# afficher les variables selectionnees

coef(modselect,1:6)

### Selection forward: autre méthode pour afficher les variables selectionnees
varselect2=summary(modselect)$which

mod2=lm(cholesterol~.,data=liver[,varselect2[6,]])
summary(mod2)


In [ ]:

selectforward=which(varselect1[6,]==TRUE)
attributes(selectforward)

selectseqrep=which(varselect2[6,]==TRUE)
attributes(selectseqrep)

In [ ]:

library(gplots)

data=list(X=selectforward,Y=selectseqrep)

v=venn(data)
attr(v, "intersections")


** Q ** Commenter ces résultats. Que concluez vous ? 

In [ ]:
a=selectforward[-1]
a
gene1=gene[,a-1]
gene1

b=selectseqrep[-1]
b
gene2=gene[,b-1]
gene2

cor(gene1,gene2)


cor(cholesterol,gene1)
cor(cholesterol,gene2)


# Sélection de modèle par pénalisation Lasso avec la  Librairie glmnet

L'utilisation de la librairie **`glmnet`** fournit des résultats plus rapides, ce qui peut s'avérer
important pour des données de grande dimension. 


## Construction du modèle

In [ ]:
library(glmnet)

x<-as.matrix(liver[,-1])
y<-liver$cholesterol
lasso<-glmnet(x,y)

par(mfrow=c(1,2))
plot(lasso,xvar="lambda")
plot(lasso,xvar="norm")

a=cv.glmnet(x,y)

lambda.opt=a$lambda.min
lambda.opt

app=glmnet(x,y,lambda=lambda.opt)
app


## Liste des variables sélectionnées : 

In [ ]:
### Lasso: variables selectionnees pour la valeur de lambda sélectionnée par validation croisée

listevar<-predict(lasso,s=lambda.opt,type="nonzero")# liste des variables avec un coefficient non-nul, pour un lambda donne
colnames(liver)[listevar[1:(dim(listevar)[1]),]+1]# liste des noms de ces variables selectionnees


## Modèle sélectionné

In [ ]:
geneselect=liver[,listevar[1:(dim(listevar)[1]),]+1]
cholesterol=liver$cholesterol
liverselect=data.frame(cholesterol,geneselect)
modselect=lm(cholesterol~.,data= liverselect)
summary(modselect)

In [ ]:
colnames(geneselect)
cor(cholesterol,geneselect)

# Sélection de modèle par pénalisation avec Elastic Net

In [ ]:
# on peut faire varier avec le paramètre alpha de glmnet
out.elnet <- glmnet(x,y,alpha=0.5)

a.elnet=cv.glmnet(x,y,alpha=0.5)
lambda.opt=a.elnet$lambda.min
app=glmnet(x,y,lambda=lambda.opt)


app

listevar<-predict(app,s=lambda.opt,type="nonzero")# liste des variables avec un coefficient non-nul, pour un lambda donne
colnames(liver)[listevar[1:(dim(listevar)[1]),]+1]# liste des noms de ces variables selectionnees


In [ ]:
a=colnames(geneselect)
b=colnames(liver)[listevar[1:(dim(listevar)[1]),]+1]
a
b

data=list(X=a,Y=b)

v=venn(data)
attr(v, "intersections")


** Q ** Commenter les résultats obtenus


# Régression PLS

In [ ]:

library(pls) 
#nombre optimal de composantes par validation croisée
simpls= mvr(cholesterol~.,data=liver, ncomp=20, validation="CV", method="simpls")
summary(simpls)

#noter le nombre optimal de composantes 
plot(simpls,"val")



In [ ]:
#Calcul des prévisions

predapp.pls=predict(simpls,ncomp=10)
resapp.pls=predapp.pls-cholesterol

#Erreur d'apprentissage
mean(resapp.pls**2)

#graphique
plot(predapp.pls,cholesterol)
abline(0,1)

** Q **  La méthode ne permet pas directement de selectionner les variables : on peut  étudier les corrélations des composantes PLS avec les variables initiales. Il est possible aussi d'utiliser la "sparse PLS". 